### Note: Please ensure that this notebook is run in the daq environment

In [1]:
import glob
import os
import tqdm
from collections import defaultdict
import re
import numpy as np
import pandas as pd
from scipy.io import wavfile
# from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.video.io.VideoFileClip import VideoFileClip
from datetime import datetime, timedelta
import copy

In [2]:
# def seconds_to_hms(seconds):
#     # Calculate hours, minutes, and seconds
#     hours, remainder = divmod(seconds, 3600)
#     minutes, seconds = divmod(remainder, 60)
#     # Format the time in HH:MM:SS
#     return f"{int(hours):02}:{int(minutes):02}:{int(seconds):02}.{int(remainder%1*1000):03}"

In [3]:
from moviepy.tools import subprocess_call
from moviepy.config import get_setting

# def ffmpeg_extract_subclip(filename, t1, t2, targetname=None):
def ffmpeg_extract_subclip(filename, start_frame, end_frame, targetname=None):
    """ Makes a new video file playing video file ``filename`` between
    the times ``t1`` and ``t2``. """
    # !ffmpeg -i {filename} -ss {seconds_to_hms(t1)} -to {seconds_to_hms(t2)} -c copy {targetname}
    tmp_str = f"between(n\,{start_frame}\,{end_frame}),setpts=PTS-STARTPTS"
    !ffmpeg -i {filename} -vf select={tmp_str} -c:v libx264 -crf 0 {targetname}


In [4]:
experiment_no = 453
no_channels = 13
camera_fps = 30
# camera_corrected_sample_time = 0.03338542
file_length = 120 # in seconds (120 sec - 2 mins)
sampling_rate = 125000
# nidaq_folder = "D:/big_setup/experiment_{}/nidaq/".format(experiment_no)
nidaq_folder = "D:/big_setup/experiment_{}/nidaq/".format(experiment_no)
video_folder = "D:/big_setup/experiment_{}/videos/".format(experiment_no)

In [5]:
# Deleting the trunc files if it is already present
trunc_files = glob.glob(nidaq_folder+"*_trunc*")
for i in trunc_files:
    os.remove(i) 

trunc_files = glob.glob(video_folder+"*_trunc*")
for i in trunc_files:
    os.remove(i) 

In [6]:
nidaq_files = glob.glob(nidaq_folder+"*")
video_files = glob.glob(video_folder+"*")

In [7]:
# Creating separate folders for the concatenated data
path_concat = "D:/big_setup/experiment_{}/concatenated_data_cam_mic_sync/temp/".format(experiment_no)

try:
    os.makedirs(path_concat)
except:
    pass

In [8]:
path_final = "D:/big_setup/experiment_{}/concatenated_data_cam_mic_sync/".format(experiment_no)

In [9]:
# Functions to sort file names correctly
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [10]:
nidaq_files.sort(key=natural_keys)
video_files.sort(key=natural_keys)

In [11]:
nidaq_files = [i.replace("\\",'/') for i in nidaq_files]
video_files = [i.replace("\\",'/') for i in video_files]

### For Videos

In [12]:
# Getting the camera names
temp_cam_name = []
for i in video_files:
    temp_cam_name.append(i.split("/")[-1].split("-")[0])
camera_names = list(set(temp_cam_name))
print(camera_names)

['gily_center']


In [13]:
# Creating a dictionary of all videos of a single camera
video_by_camera = defaultdict(lambda:[])
for idx,name in enumerate(temp_cam_name):
    video_by_camera[name].append(video_files[idx])

In [14]:
cam_clk_data = pd.read_csv(f"D:/big_setup/experiment_{experiment_no}/camera_timestamps.csv")

In [15]:
file_breaks  = np.arange(0,cam_clk_data.tail(1).index[0]/camera_fps,file_length)
last_break = cam_clk_data.tail(1).index[0]
timestamp_idx = []
for val in file_breaks:
    timestamp_idx.append((np.abs(cam_clk_data[f'concat_{camera_names[0]}_time_from_vid_start'] - val)).argmin())

In [16]:
if timestamp_idx[-1] != last_break:
    timestamp_idx.append(last_break)

### Assumes 
- 5 min original videos and shorter concatenated videos
- concatenation starts from 0.0 of first video for each camera  
- all videos are recorded properly

In [17]:
vals = []
for cam in camera_names:
    file_concat_index = 0 
    no_videos = len(video_by_camera[cam])
    id = 0
    
    temp = cam_clk_data.iloc[timestamp_idx[0]][f'{cam}_file_name'].split('\\')
    prev_vid_name = temp[0]+'/'+temp[1]
    # prev_break_time = cam_clk_data.iloc[timestamp_idx[0]][f'{cam}_time_from_vid_start']
    prev_break_frame = cam_clk_data.iloc[timestamp_idx[0]][f'{cam}_frame_idx']
    prev_file_break = timestamp_idx[0]

    for file_break in timestamp_idx[1:]:

            #--------------------------------------- edited -----------------------------------------------

            temp = cam_clk_data.iloc[file_break][f'{cam}_file_name'].split('\\')
            vid_name = temp[0]+'/'+temp[1]
            # break_time = cam_clk_data.iloc[file_break][f'{cam}_time_from_vid_start']
            break_frame = cam_clk_data.iloc[file_break][f'{cam}_frame_idx']
            str_1 = "%03d"%(file_concat_index)
            filename_txt = path_concat + f"video_{cam}_{str_1}.txt"

            if prev_vid_name != vid_name: # when we reach the end of a saved file and start a new one

                with VideoFileClip(prev_vid_name) as video:
                    # end_time = video.duration
                    end_frame = video.reader.nframes
                    
                    # vals.append([end_time,prev_break_time])
                    # if end_time-prev_break_time > 1:
                    if end_frame-prev_break_frame > 1:
                        tmp = cam_clk_data.iloc[prev_file_break][f'{cam}_file_name'].split('\\')
                        target_file = tmp[0]+"/"+tmp[1].split(".")[0]+f"_trunc_{id}.mp4"      
                        # ffmpeg_extract_subclip(prev_vid_name, prev_break_time, end_time, targetname=target_file)
                        ffmpeg_extract_subclip(prev_vid_name, prev_break_frame, end_frame-1, targetname=target_file)
                
                        f = open(filename_txt, 'a')
                        f.write("file \'{}\'\n".format(target_file))
                        f.close()
                    
                    # prev_break_time = 0.0
                    prev_break_frame = 0
                    id = 0
                    
            target_file = temp[0]+"/"+temp[1].split(".")[0]+f"_trunc_{id}.mp4"
            # ffmpeg_extract_subclip(vid_name, prev_break_time, break_time, targetname=target_file)
            ffmpeg_extract_subclip(vid_name, prev_break_frame, break_frame-1, targetname=target_file)

            f = open(filename_txt, 'a')
            f.write("file \'{}\'\n".format(target_file))
            f.close()
            
            # prev_break_time = copy.deepcopy(break_time)
            prev_break_frame =  copy.deepcopy(break_frame)
            id+=1
            file_concat_index+=1
            prev_vid_name = copy.deepcopy(vid_name)
            prev_file_break = copy.deepcopy(file_break)

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 10.2.1 (GCC) 20200726
  configuration: --disable-static --enable-shared --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libsrt --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libgsm --enable-librav1e --disable-w32threads --enable-libmfx --enable-ffnvcodec --enable-cuda-llvm --enable-cuvid --enable-d3d11va 

### For NIDAQ

In [18]:
# Reading the timestamps data for the camera clock channel 
audio_clk_data = pd.read_csv(f"D:/big_setup/experiment_{experiment_no}/camera_timestamps.csv")
audio_start_rec_idx = int(cam_clk_data["clk_ch_file_name"][0].split("\\")[1].split("_")[2])
audio_stop_rec_idx = int(cam_clk_data["clk_ch_file_name"].iloc[-1].split("\\")[1].split("_")[2])
start_sample_index =  int(cam_clk_data["clk_ch_sample_idx"][0])
end_sample_index =  int(cam_clk_data["clk_ch_sample_idx"].iloc[-1])

In [19]:
# Concatenating the microphone channels
for ch_no in range(no_channels):
    file_concat_index = 0 
    id = 0

    prev_ch_name = "_".join(audio_clk_data.iloc[timestamp_idx[0]][f'clk_ch_file_name'].split("_")[:-1])+f"_{ch_no}.wav"
    prev_break_idx = audio_clk_data.iloc[timestamp_idx[0]][f'clk_ch_sample_idx']


    for file_break in tqdm.tqdm(timestamp_idx[1:]):

            #--------------------------------------- edited -----------------------------------------------

            ch_name = "_".join(audio_clk_data.iloc[file_break][f'clk_ch_file_name'].split("_")[:-1])+f"_{ch_no}.wav"
            break_idx = audio_clk_data.iloc[file_break][f'clk_ch_sample_idx']
            str_1 = "%02d"%(ch_no)
            str_2 = "%03d"%(file_concat_index)
            filename_txt = path_concat + f"channel_{str_1}_file_{str_2}.txt"

            if prev_ch_name != ch_name: # when we reach the end of a saved file and start a new one
                    if prev_break_idx < (sampling_rate*300)-1000:
                        target_file = prev_ch_name.split(".")[0]+f"_trunc_{id}.wav"   

                        # Loading the file
                        sampling_rate,ch = wavfile.read(prev_ch_name)
                        ch = ch[prev_break_idx:]
                        
                        wavfile.write(target_file,sampling_rate,ch)
                
                        f = open(filename_txt, 'a')
                        f.write("file \'{}\'\n".format(target_file))
                        f.close()
                    
                    prev_break_idx = 0
                    id = 0
                    
            target_file = ch_name.split(".")[0]+f"_trunc_{id}.wav"

            # Loading the file
            sampling_rate,ch = wavfile.read(ch_name)
            ch = ch[prev_break_idx:break_idx]
                        
            wavfile.write(target_file,sampling_rate,ch)
                
            f = open(filename_txt, 'a')
            f.write("file \'{}\'\n".format(target_file))
            f.close()
                    

            
            prev_break_idx = copy.deepcopy(break_idx)
            id+=1
            file_concat_index+=1
            prev_ch_name = copy.deepcopy(ch_name)

100%|██████████| 19/19 [00:07<00:00,  2.63it/s]


## Steps done via code in the below cell 
- open anaconda prompt
- Enter the following commands:
    - For video concatenation:
        - ```conda activate daq```
        - ```ffmpeg -f concat -safe 0 -i {path_to_text_file} -c copy {path_to_output_file}```
            
            eg: ```ffmpeg -f concat -safe 0 -i D:\big_setup\experiment_9\concatenated_data_cam_mic_sync\video_e3v83b3.txt -c copy D:\big_setup\experiment_9\concatenated_data_cam_mic_sync\e3v83b3_ffmpeg.mp4```
    - For NIDAQ concatenation:
        - - ```conda activate daq```
        - ```ffmpeg -f concat -safe 0 -i {path_to_text_file} -c copy {path_to_output_file}```
            
            eg: ```ffmpeg -f concat -safe 0 -i D:\big_setup\experiment_9\concatenated_data_cam_mic_sync\channel_0.txt -c copy D:\big_setup\experiment_9\concatenated_data_cam_mic_sync\channel_0_ffmpeg.wav```

In [20]:
no_audio_files = int(len(glob.glob(path_concat + "*channel_*.txt"))/no_channels)
no_video_files = int(len(glob.glob(path_concat + "*video_*.txt"))/len(camera_names))

In [21]:
timestamp_record_data = cam_clk_data.iloc[timestamp_idx]
time_starts = []
time_ends = []
column_names = [i for i in timestamp_record_data.columns if i.endswith("file_name") and not(i.startswith('clk'))]

starting_timestamp = datetime.strptime(timestamp_record_data[column_names[0]][0].split('\\')[-1].split('-')[1], '%Y%m%dT%H%M%S')

# for camera_file in column_names:
#     for file_name in timestamp_record_data[camera_file]:
#         time_starts.append(datetime.strptime(file_name.split('\\')[-1].split('-')[1], '%Y%m%dT%H%M%S'))
#         time_ends.append(time_starts[-1] + timedelta(minutes=file_length/60))

In [22]:
# if no_audio_files != no_video_files:
#     raise Exception("Number of audio and video files don't match")
# else:
matched_csv = []
timestamp_write = starting_timestamp
f = open(path_final+"README.txt", 'w')
f.write("The files that are synced are as follows:\n")
f.write("Video file names - corresponding audio file names - timestamp range of file \n")
for i in range(no_audio_files):
    temp_0 = []
    temp_1 = []
    temp_2 = [timestamp_write.strftime("%Y-%m-%d %H:%M:%S"),(timestamp_write + timedelta(minutes=file_length/60)).strftime("%Y-%m-%d %H:%M:%S")]
    timestamp_write += timedelta(minutes=file_length/60)
    for j in range(no_channels):
        var_1 = "%03d"%(i)
        var_2 = "%02d"%(j)
        temp_0.append(f"channel_{var_2}_file_{var_1}")
    for j in camera_names:
        var_1 = "%03d"%(i)
        temp_1.append(f"video_{j}_{var_1}")
    f.write(f"[{', '.join(temp_1)}] - [{', '.join(temp_0)}] - [{', '.join(temp_2)}]\n")
    matched_csv.append([temp_1,temp_0,temp_2])
pd.DataFrame(matched_csv,columns=["video","audio","timestamp"]).to_csv(path_final+"sync.csv", index = False)
f.close()

In [23]:
filename_txt = path_concat + "*.txt"
txt_files = glob.glob(filename_txt)
video_files = []
audio_files = []

for file in txt_files:
    name = file.split('\\')[-1].split(".")[0]
    full_path_txt = path_concat+name+".txt"
    full_path_txt = full_path_txt.replace("/","\\")
    if name[0] == "c":
        full_path_wav = path_final+name+".wav"
        full_path_wav = full_path_wav.replace("/","\\")
        !ffmpeg -f concat -safe 0 -i {full_path_txt} -c copy  {full_path_wav}
        # -rf64 auto
    else:
        full_path_mp4 = path_final+name+".mp4"
        full_path_mp4 = full_path_mp4.replace("/","\\")
        !ffmpeg -f concat -safe 0 -i {full_path_txt} -c copy {full_path_mp4}

ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 10.2.1 (GCC) 20200726
  configuration: --disable-static --enable-shared --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libsrt --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libgsm --enable-librav1e --disable-w32threads --enable-libmfx --enable-ffnvcodec --enable-cuda-llvm --enable-cuvid --enable-d3d11va 

## Testing sync

In [ ]:
# import warnings
# import math

In [ ]:
# # Redoing the files that didn't get written
# channel_txt_search = path_concat + "*channel_*.txt"
# video_txt_search = path_concat + "*video_*.txt"

# channel_txt_filenames = glob.glob(channel_txt_search)
# video_txt_filenames = glob.glob(video_txt_search)

# channel_txt_filenames.sort(key=natural_keys)
# video_txt_filenames.sort(key=natural_keys)

# upper_no_audio_files = math.ceil(len(channel_txt_filenames)/no_channels)
# upper_no_video_files = math.ceil(len(video_txt_filenames)/len(camera_names))
# print(f"Number of audio files: {upper_no_audio_files}")
# print(f"Number of video files: {upper_no_video_files}")



# # For channel txt files
# prev_ch_idx_nos = 0
# prev_ch_nos = 0

# # if not(len(channel_txt_filenames)%no_channels > (no_channels-1)):
# #     raise Exception(f"Channel txt files are missing re-run code")


# # First test - no of indices per channel is the same
# if len(channel_txt_filenames)%no_channels > 0:
#     warnings.warn("The length of the audio channel text files do not match")
#     print("Testing each channels txt files")

#     counter = 0

#     for name in channel_txt_filenames:
#         vals = name.split("\\")[-1].split('_')
#         channel_ = int(vals[1])
#         index_ = int(vals[2].split(".")[0])

#         if channel_ - prev_ch_nos == 1:
#                 prev_ch_nos = channel_
#                 prev_ch_idx_nos = 0
#                 counter = 0
#         else:
#             raise Exception(f"Missing channel {prev_ch_nos+1} - need to rerun code")

#         if channel_ - prev_ch_nos == 0:
#             if counter == 0:
#                 counter = 1
#                 if  index_ != prev_ch_idx_nos:
#                     raise Exception(f"Missing channel {prev_ch_nos} index {prev_ch_idx_nos} - need to rerun code")
                
#             if prev_ch_idx_nos - index_ > 1:
#                     no_missing_files =  prev_ch_idx_nos - index_
#                     raise Exception(f"There are {no_missing_files} text files in channel {channel_} between {prev_ch_idx_nos} and {index_} - need to rerun code") 

            
#             prev_ch_idx_nos+=1

        
#         temp = file.split('\\')[-1].split(".")[0]
#         full_path_txt = path_concat+temp+".txt"
#         full_path_txt = full_path_txt.replace("/","\\")
#         full_path_wav = path_final+name+".wav"
#         full_path_wav = full_path_wav.replace("/","\\")
#         if  os.path.exists(full_path_wav):
#             continue
#         else:
#             warnings.warn(f"File {temp}.wav doesn't exist - trying to rerun")
#             !ffmpeg -f concat -safe 0 -i {full_path_txt} -c copy  {full_path_wav}
#             if not(os.path.exists(full_path_wav)):
#                    raise Exception("Unable to create file from txt, re run concat")
        
    

# # For video txt files 
# prev_vid_idx_nos = 0
# camera_names.sort(key=natural_keys)
# prev_vid_name = camera_names[0]


# # if not(len(video_txt_filenames)/len(camera_names) > len(camera_names)-1):
# #     raise Exception(f"Video txt files are missing re-run code")

# # First test - no of indices per video is the same
# if len(video_txt_filenames)//len(camera_names) != len(channel_txt_filenames)//no_channels:
#     warnings.warn("The length of the video text files per camera do not match")
#     print("Testing each cameras txt files")

#     counter = 0

#     for name in video_txt_filenames:

#         vals = name.split("\\")[-1].split('_')
#         video_ = vals[1]
#         index_ = int(vals[2].split(".")[0])

#         if video_ != prev_vid_name:
#                 prev_vid_name = video_
#                 prev_vid_idx_nos = 0
#                 counter = 0

#         if video_ == prev_vid_name:
#             if counter == 0:
#                 counter = 1
#                 if  index_ != prev_vid_idx_nos:
#                     raise Exception(f"Missing video {prev_vid_name} index {prev_vid_idx_nos}")
                
#             if prev_vid_idx_nos - index_ > 1:
#                     no_missing_files =  prev_vid_idx_nos - index_
#                     raise Exception(f"There are {no_missing_files} text files in channel {video_} between {prev_vid_idx_nos} and {index_}") 

            
#             prev_ch_idx_nos+=1


#         temp = file.split('\\')[-1].split(".")[0]
#         full_path_txt = path_concat+temp+".txt"
#         full_path_txt = full_path_txt.replace("/","\\")

#         full_path_mp4 = path_final+name+".mp4"
#         full_path_mp4 = full_path_mp4.replace("/","\\")

#         if  os.path.exists(full_path_mp4):
#             continue
#         else:
#             warnings.warn(f"File {temp}.mp4 doesn't exist - trying to rerun")
#             !ffmpeg -f concat -safe 0 -i {full_path_txt} -c copy  {full_path_mp4}
#             if not(os.path.exists(full_path_wav)):
#                    raise Exception("Unable to create file from txt, re run concat")

Number of audio files: 51
Number of video files: 51


old camera frame rate - 1/0.03338542

new camera frame rate - 